**Convolutional Neural Network**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
heart.csv
df = pd.read_csv('')

from sklearn.model_selection import train_test_split

X = df.drop("target",axis=1).values
y = df["target"].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=101)

/Users/jaydip/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [3]:
#scaling data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(212, 13, 1)
X_test = X_test.reshape(91, 13, 1)

In [4]:
X_train.shape

(212, 13, 1)

In [5]:
X_test.shape

(91, 13, 1)

In [6]:
y_train.shape

(212,)

In [7]:
y_test.shape

(91,)

In [8]:
from tensorflow.keras.utils import to_categorical

y_cat_test = to_categorical(y_test)
y_cat_train = to_categorical(y_train)

/Users/jaydip/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jaydip/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jaydip/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jaydip/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Flatten, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import Dropout, LSTM, Embedding
from keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

Using TensorFlow backend.


In [10]:
classifier = Sequential()

classifier.add(Conv1D(256, 20, input_shape = (13,1), activation = 'relu', padding='same'))
classifier.add(MaxPooling1D(pool_size = 1))
classifier.add(Dropout(0.2))

classifier.add(Conv1D(256, 20, activation = 'relu', padding='same', input_shape = (13,1)))
classifier.add(MaxPooling1D(pool_size = 1))
classifier.add(Dropout(0.2))

classifier.add(Conv1D(256, 20, activation = 'relu', padding='same', input_shape = (13,1)))
classifier.add(MaxPooling1D(pool_size = 1))
classifier.add(Dropout(0.2))

classifier.add(LSTM(20, activation = 'relu'))
classifier.add(Dropout(0.2))

classifier.add(Dense(activation = 'relu', units=1024))
classifier.add(Dropout(0.3))

classifier.add(Dense(activation = 'relu', units=1024))
classifier.add(Dropout(0.3))

classifier.add(Dense(activation = 'relu', units=1024))
classifier.add(Dropout(0.3))

classifier.add(Dense(activation = 'relu', units=1024))
classifier.add(Dropout(0.3))

classifier.add(Dense(activation = 'sigmoid', units=2))

classifier.compile(optimizer = Adam(lr=0.00001), loss = 'categorical_crossentropy')
print(classifier.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 13, 256)           5376      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 13, 256)           0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 13, 256)           1310976   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)  

In [11]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [12]:
classifier.fit(X_train, y_cat_train, epochs=500, batch_size=20, validation_split = 0.1,callbacks=[early_stop])

Train on 190 samples, validate on 22 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
190/190 [==============================] - 4s 21ms/sample - loss: 0.6931 - val_loss: 0.6930
Epoch 2/500
190/190 [==============================] - 2s 9ms/sample - loss: 0.6931 - val_loss: 0.6929
Epoch 3/500
190/190 [==============================] - 2s 9ms/sample - loss: 0.6931 - val_loss: 0.6927
Epoch 4/500
190/190 [==============================] - 2s 9ms/sample - loss: 0.6930 - val_loss: 0.6926
Epoch 5/500
190/190 [==============================] - 2s 9ms/sample - loss: 0.6930 - val_loss: 0.6925
Epoch 6/500
190/190 [==============================] - 3s 14ms/sample - loss: 0.6929 - val_loss: 0.6922
Epoch 7/500
190/190 [==============================] - 2s 10ms/sample - loss: 0.6929 - val_loss: 0.6920
Epoch 8/500
190/190 [==============================] - 2s 13ms/sample - loss: 0.6928 - val_loss: 0.6917
Epoch 9/500
190/190 [==============================] - 2s 11ms/sample - loss: 0

In [13]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

nn_pred = classifier.predict_classes(X_test)
print(classification_report(y_test,nn_pred))
print(confusion_matrix(y_test,nn_pred))

              precision    recall  f1-score   support

           0       0.87      0.75      0.80        44
           1       0.79      0.89      0.84        47

    accuracy                           0.82        91
   macro avg       0.83      0.82      0.82        91
weighted avg       0.83      0.82      0.82        91

[[33 11]
 [ 5 42]]
